In [2]:
import random
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from PIL import Image
import itertools

class SiameseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_folder = datasets.ImageFolder(root=root_dir)
        self.transform = transform
        self.image_pairs = list(itertools.combinations_with_replacement(range(len(self.image_folder)), 2))
        self.targets = [int(self.image_folder.targets[idx1] == self.image_folder.targets[idx2]) for idx1, idx2 in self.image_pairs]

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, index):
        idx1, idx2 = self.image_pairs[index]
        img1,_ = self.image_folder[idx1]
        img2,_ = self.image_folder[idx2]
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, self.targets[index]

/opt/homebrew/Caskroom/miniconda/base/envs/oneshot-face/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/oneshot-face/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2D1B8D5C-7891-3680-9CF9-F771AE880676> /opt/homebrew/Caskroom/miniconda/base/envs/oneshot-face/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <86F5F3CA-817C-3EB3-840D-884F6BAEC4B8> /opt/homebrew/Caskroom/miniconda/base/envs/oneshot-face/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
import torch
from torch.utils.data import Sampler
import numpy as np
from collections import Counter

class RandomUnderSampler(Sampler):
    def __init__ (self, targets, seed=None, shuffle=False):
        self.targets = np.array(targets)
        self.class_counts = Counter(self.targets)
        self.classes = self.class_counts.keys()
        self.indices = {cls : np.where(self.targets==cls)[0] for cls in self.classes}
        self.seed = seed
        self.min_count = min(self.class_counts.values())
        self.shuffle = shuffle

    def __iter__(self):
        sampled_indices = []
        for cls, indices in self.indices.items():
            if self.seed is not None:
                np.random.seed(self.seed)
            sampled_indices.extend(np.random.choice(indices, self.min_count))
        if self.shuffle:
            np.random.shuffle(sampled_indices)
        return iter(sampled_indices)
    
    def __len__(self):
        return self.min_count * len(self.classes)
        
        

In [4]:
ds = SiameseDataset('data/raw', transform=transforms.ToTensor())

In [5]:
sampler = RandomUnderSampler(ds.targets)

In [6]:
dl = torch.utils.data.DataLoader(ds, sampler=sampler, batch_size=510980)

In [7]:
targets = []
indices = []
for idx in sampler:
    targets.append(ds.targets[idx])
    indices.append(idx)
print(np.bincount(targets))
print(indices[0])

[255490 255490]
44767913


In [8]:
import numpy as np

In [9]:
class_count = np.bincount(ds.targets)

In [10]:
class_count

array([87307271,   255490])

In [11]:
from collections import Counter

In [12]:
class_count = Counter(ds.targets)

In [13]:
class_count

Counter({0: 87307271, 1: 255490})

In [14]:
class_count.keys()

dict_keys([1, 0])

In [15]:
minority_class_count = min(class_count.values())
minority_class_count

255490

In [16]:
for i in class_count.items():
    print(i)

(1, 255490)
(0, 87307271)


In [17]:
print([item[0] for item in class_count.items()])

[1, 0]


In [19]:
indices = {cls: np.where(ds.targets == cls) for cls in class_count}
indices

/var/folders/1n/7gb9mkld76351c83y6pwmhhr0000gn/T/ipykernel_42202/3412869768.py:1: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  indices = {cls: np.where(ds.targets == cls) for cls in class_count}


{1: (array([], dtype=int64),), 0: (array([], dtype=int64),)}

In [21]:
np.unique(ds.targets)

array([0, 1])

In [22]:
len(ds.targets)

87562761

In [23]:
targets = np.array(ds.targets)
targets

array([1, 0, 0, ..., 1, 0, 1])

In [24]:
np.where(targets==0)[0]

array([       1,        2,        3, ..., 87562756, 87562757, 87562759])

In [25]:
classes = np.unique(ds.targets)
classes

array([0, 1])

In [26]:
indices_per_class = {cls: np.where(targets == cls)[0] for cls in classes}
indices_per_class

{0: array([       1,        2,        3, ..., 87562756, 87562757, 87562759]),
 1: array([       0,    13233,    26465, ..., 87562755, 87562758, 87562760])}

In [27]:
sampled_indices = []

In [28]:
min_count  = min(class_count.values())
min_count

255490

In [29]:
for cls, indices in indices_per_class.items():
    sampled_indices.extend(np.random.choice(indices,min_count))

In [30]:
len(sampled_indices)

510980

In [58]:
torch.nn.Sigmoid

torch.nn.modules.activation.Sigmoid

In [31]:
from torchvision.models import mobilenetv2

In [32]:
model = mobilenetv2.MobileNetV2()

In [33]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [34]:
batch_size = 1
channels = 3
height = 224
width = 224

In [35]:
dummy_input = torch.randn(batch_size, channels, height, width)

In [47]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [50]:
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [41]:
dummy_input2 = model.features(dummy_input[:1])

In [42]:
dummy_input2.shape

torch.Size([1, 1280, 7, 7])

In [55]:
avgpool = torch.nn.AdaptiveAvgPool2d(1)

In [56]:
dummy_input3 = avgpool(dummy_input2)
dummy_input3.shape

torch.Size([1, 1280, 1, 1])

In [57]:
torch.onnx.export(model.classifier,dummy_input3,'model_2.onnx')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1280x1 and 1280x1000)

In [61]:
from torch import nn
from torchvision.models.mobilenetv2 import MobileNetV2

In [73]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.backbone = MobileNetV2()
        self.fc = nn.Sequential(
            nn.Linear(1280,1280),
            nn.Sigmoid(),
        )

    def forward_one_branch(self, x):
        x = self.backbone.features(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1,1))
        x = torch.flatten(x)
        x = self.fc(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one_branch(input1)
        output2 = self.forward_one_branch(input2)
        return output1, output2

In [77]:
import torch.nn.functional as F

In [ ]:
class EuclideanDistance(nn.Module):
    def __init__(self):
        super(EuclideanDistance, self).__init__()
    
    def forward(self, output1, output2):
        return torch.sqrt(torch.sum((output1-output2)**2))


In [85]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, distance, label):
        loss_negative = (1 - label) * torch.pow(distance, 2)
        loss_positive = label * torch.pow(torch.clamp(self.margin - distance, min=0.0), 2)
        loss = torch.mean(loss_negative + loss_positive)
        return loss

In [78]:
dummy1 = torch.tensor([1,1,1,1,1,1,1])
dummy2 = torch.tensor([1,1,1,1,1,1,1])

In [87]:
loss = ContrastiveLoss()
loss(torch.tensor(0.8),torch.tensor(0))

tensor(0.6400)

In [81]:
(dummy1-dummy2)**2

tensor([0, 0, 0, 0, 0, 0, 0])

In [84]:
torch.clamp(-1,0)

TypeError: clamp() received an invalid combination of arguments - got (int, int), but expected one of:
 * (Tensor input, Tensor min = None, Tensor max = None, *, Tensor out = None)
 * (Tensor input, Number min = None, Number max = None, *, Tensor out = None)


In [79]:
torch.sum(dummy1-dummy2)

tensor(0)

In [71]:
embedding_layer = Embedding()

In [72]:
output1, output2 = embedding_layer(dummy_input, dummy_input)
print(output1.shape, output2.shape)

torch.Size([1280]) torch.Size([1280])


In [75]:
torch.max(output1)

tensor(0.6835, grad_fn=<MaxBackward1>)

In [76]:
torch.max(output2)

tensor(0.6835, grad_fn=<MaxBackward1>)